# Backtest

In [ ]:
from system.agent import Agent
from system.env import Env

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2


config = {"capital" : 100, "interval" : "1d",
          "symbols" : ["BTC", "ETH", "SOL", "DASH", "MATIC",
                       "TWT", "GALA", "EGLD", "XMR"],
          "start" : "2019", "end" : "2023"
          }

env = Env(**config)

agentIds = [1, 2, 3, 4]
env.config_agents(agentIds)

params_1 = {"agentId" : 1, "symbol" : "BTC", "allocation" : 100, "env" : env, "policy_name" : "MOMENTUM"}
params_2 = {"agentId" : 2, "symbol" : "BTC", "allocation" : 100, "env" : env, "policy_name":"TRIPLEMA"}
params_3 = {"agentId" : 3, "symbol" : "ETH", "allocation" : 100, "env" : env, "policy_name":"MOMENTUM"}
params_4 = {"agentId" : 4, "symbol" : "ETH", "allocation" : 100, "env" : env, "policy_name":"TRIPLEMA"}

In [ ]:
agent_1 = Agent(**params_1)
agent_1.update_policy_params(params = 3)
agent_1.run_episode()

agent_2 = Agent(**params_2)
agent_2.update_policy_params(params = (7, 14, 21))
agent_2.run_episode()

agent_3 = Agent(**params_3)
agent_3.update_policy_params(params = 5)
agent_3.run_episode()

agent_4 = Agent(**params_4)
agent_4.update_policy_params(params = (7, 14, 21))
agent_4.run_episode()

In [ ]:
import datetime

date = "2019-02-14 00:00:00"
d2 = datetime.datetime.strptime(date, "%Y-%m-%d %H:%M:%S")
d2

In [ ]:
d = env.journal.tradesData.date.iloc[0]
d

In [ ]:
d == d2

In [ ]:
env.get_viz(agentId=1, symbol = "BTC")

## Global Report

In [ ]:
env.globalReport()

In [ ]:
d = {"a":1, "b":2, "c":3}
r = {}
t = {x : d.pop(x, None) for x in ["a", "b", "c"]}
t

In [ ]:
d

#  MultiAgent System v2

In [ ]:
from magent import MAgentThread
from master import  MasterAgentThread
from system.env import Env

from threading import Thread, Condition, Event, Barrier, Semaphore

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

START = "2023-01-01 00:00:00"
END = "2023-10-01 00:00:00"
config = {"capital" : 100, "interval" : "1d",
          "symbols" : ["BTC", "ETH", "SOL", "DASH", "MATIC",
                       "TWT", "GALA", "EGLD", "XMR"],
          "start" : "2023", "end" : "2023"
          }
env = Env(**config)


def custom_action():
    print(" //////////////// *** Barrier passee *** \\\\\\\\\\\\\\\\")


agentId_1 = 1
agentId_2 = 2
agentId_3 = 3
agentId_4 = 4
agentID = [1, 2, 3, 4]
env.config_agents(agentID)

agent_bus = {
    "fstep" : {agentId_1 : False, agentId_2 : False, agentId_3 : False, agentId_4 : False},
    "msg" : {},
    "stop" : False
}
master_bus = {
    "msg" : {},
    "activeAgent" : None,
    "stop" : False
}

activeAgent = agentId_1

N = len(agentID)
condition = Condition()
barrier = Barrier(N, action=custom_action)

master_params = {"condition" : condition, "agent_bus" : agent_bus, "master_bus" : master_bus}

params_1 = {"agentId" : agentId_1, "symbol" : "BTC", "allocation" : 100, "env" : env, "policy_name" : "MOMENTUM",
            "condition" : condition, "barrier" : barrier, "agent_bus" : agent_bus, "master_bus" : master_bus}

params_2 = {"agentId" : agentId_2, "symbol" : "ETH", "allocation" : 100, "env" : env, "policy_name":"TRIPLEMA",
            "condition" : condition, "barrier" : barrier, "agent_bus" : agent_bus, "master_bus" : master_bus}

params_3 = {"agentId" : agentId_3, "symbol" : "SOL", "allocation" : 100, "env" : env, "policy_name" : "MOMENTUM",
            "condition" : condition, "barrier" : barrier, "agent_bus" : agent_bus, "master_bus" : master_bus}

params_4 = {"agentId" : agentId_4, "symbol" : "EGLD", "allocation" : 100, "env" : env, "policy_name":"TRIPLEMA",
            "condition" : condition, "barrier" : barrier, "agent_bus" : agent_bus, "master_bus" : master_bus}


master = MasterAgentThread(**master_params)
master.addAgent(agentId_1)
master.addAgent(agentId_2)
master.addAgent(agentId_3)
master.addAgent(agentId_4)

agent_1 = MAgentThread(**params_1)
agent_1.update_policy_params(params = 3)

agent_2 = MAgentThread(**params_2)
agent_2.update_policy_params(params = (7, 14, 21))

agent_3 = MAgentThread(**params_3)
agent_3.update_policy_params(params = 6)

agent_4 = MAgentThread(**params_4)
agent_4.update_policy_params(params = (14, 21, 36))


In [ ]:
master.start()
agent_1.start()
agent_2.start()
agent_3.start()
agent_4.start()

agent_1.join()
agent_2.join()
agent_3.join()
agent_4.join()
master.join()

In [ ]:
env.get_viz(agentId=1, symbol = "BTC")

In [ ]:
env.globalReport()

# *Multi-Agent* v2(1)

In [5]:
from magent import MAgentThread
from master import  MasterAgentThread
from system.env import Env
from system.menv import MEnv

from threading import Thread, Condition, Event, Barrier, Semaphore

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2


TICKERS = ["BTC", "ETH", "SOL", "DASH"]
START = "2023-01-01"
END = "2023-10-01"
INTERVAL = "1d"

CAPITAL = 100
config = {"capital" : CAPITAL, "interval" : INTERVAL,
          "symbols" : TICKERS, "start" : START, "end" : END}
env = Env(**config)


# Config menv
def set_config(symbol, capital = CAPITAL, interval = INTERVAL, start = START, end = END):
    config = {"capital" : capital, "interval" : interval,
              "symbols" : symbol, "start" : start, "end" : end}
    return config


def set_env(symbol):
    config = set_config(symbol)
    env = MEnv(**config)
    return env

menvs = [set_env(symbol) for symbol in TICKERS]


# Agent details
AGENT_PARAMS = [
    [(1, "BTC"), menvs[0]],
    [(2, "ETH"), menvs[1]],
    [(3, "SOL"), menvs[2]],
    [(4, "EGLD"), menvs[3]],
]

N = len(AGENT_PARAMS)

STRATEGIES = [
    ((1, "BTC"), "MOMENTUM", 3),
    ((2, "ETH"), "TRIPLEMA", (3, 7, 14)),
    ((3, "SOL"), "MOMENTUM", 0),
    ((4, "EGLD"), "TRIPLEMA", (14, 21, 35)),
]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
master = MasterAgentThread(env)
master.set_barrier(n=N)

for agent_param, strategie in zip(AGENT_PARAMS, STRATEGIES):
    master.addAgent(*agent_param)
    master.update_params(*strategie)

master.active_agents()
master.run()

Exception in thread Thread-13:
Traceback (most recent call last):
  File "c:\Users\Dell\miniconda3\envs\WcedSyst\Lib\threading.py", line 1038, in _bootstrap_inner
Exception in thread Thread-16:
Traceback (most recent call last):
  File "c:\Users\Dell\miniconda3\envs\WcedSyst\Lib\threading.py", line 1038, in _bootstrap_inner
Exception in thread Thread-15:
Traceback (most recent call last):
  File "c:\Users\Dell\miniconda3\envs\WcedSyst\Lib\threading.py", line 1038, in _bootstrap_inner
Exception in thread Thread-14:
Traceback (most recent call last):
  File "c:\Users\Dell\miniconda3\envs\WcedSyst\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "c:\Users\Dell\Desktop\CedAlgo\C-Syst\magent.py", line 60, in run
    self.run()
  File "c:\Users\Dell\Desktop\CedAlgo\C-Syst\magent.py", line 60, in run
    master_bus, master_status, active = self.get_master_bus()
                                        ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Dell\Desktop\CedAlgo\C-Syst\mag

[(1, 'BTC'), <system.menv.MEnv object at 0x0000017E83E18D50>, <threading.Event at 0x17e83ceb8d0: unset>, <queue.Queue object at 0x0000017E83E10710>, <queue.Queue object at 0x0000017E83E10FD0>, <threading.Barrier at 0x17e83e12e50: waiters=0/4>]
[(2, 'ETH'), <system.menv.MEnv object at 0x0000017E83E1B350>, <threading.Event at 0x17e83ceb8d0: unset>, <queue.Queue object at 0x0000017E83E110D0>, <queue.Queue object at 0x0000017E83E10450>, <threading.Barrier at 0x17e83e12e50: waiters=0/4>]
[(3, 'SOL'), <system.menv.MEnv object at 0x0000017E83D2AFD0>, <threading.Event at 0x17e83ceb8d0: unset>, <queue.Queue object at 0x0000017E83E13910>, <queue.Queue object at 0x0000017E83E11150>, <threading.Barrier at 0x17e83e12e50: waiters=0/4>]
[(4, 'EGLD'), <system.menv.MEnv object at 0x0000017E83D32910>, <threading.Event at 0x17e83ceb8d0: unset>, <queue.Queue object at 0x0000017E83E12AD0>, <queue.Queue object at 0x0000017E83E12090>, <threading.Barrier at 0x17e83e12e50: waiters=0/4>]
id :  (1, 'BTC')
(1, 'B

# *End*

In [ ]:
# Set master
master = MasterAgentThread(**master_params)

# Set agent
agent_1 = MAgentThread(**params[0])
agent_1.update_policy_params(params = 3)

agent_2 = MAgentThread(**params[1])
agent_2.update_policy_params(params = (7, 14, 21))

agent_3 = MAgentThread(**params[2])
agent_3.update_policy_params(params = 6)

agent_4 = MAgentThread(**params[3])
agent_4.update_policy_params(params = (14, 21, 36))

In [ ]:
master.start()
agent_1.start()
agent_2.start()
agent_3.start()
agent_4.start()

agent_1.join()
agent_2.join()
agent_3.join()
agent_4.join()
master.join()

In [ ]:
d = {"a":1, "b":2}
d
for x, y in d:
    print(x, y)

# *Multi-Agent v3* Multiprocessing

In [ ]:
import multiprocessing
from multiprocessing import Process, Manager, Queue, Barrier, Event, Semaphore




# Plot

In [ ]:
import plotly.graph_objects as go

categories = ['processing cost','mechanical properties','chemical stability',
              'thermal stability', 'device integration']

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
      r=[1, 5, 2, 2, 3],
      theta=categories,
      fill='toself',
      name='Product A'
))
fig.add_trace(go.Scatterpolar(
      r=[4, 3, 2.5, 1, 2],
      theta=categories,
      fill='toself',
      name='Product B'
))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 5]
    )),
  showlegend=False
)

fig.show()

In [ ]:
class MyGenerator:
    def __init__(self, n):
        self.n = n
        self.current = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.current < self.n:
            result = self.current
            self.current += 1
            return result
        else:
            raise StopIteration

# Créez une instance de la classe MyGenerator
dix_nombres = MyGenerator(10)

# Utilisez une boucle for pour obtenir les valeurs
for nombre in dix_nombres:
    print(nombre)


In [ ]:
my_list = [1, 2, 3, 4, 5]
iterable = iter(my_list)

for x in iterable:
    print(x)


In [ ]:
import multiprocessing

multiprocessing.cpu_count()

In [ ]:
import os 

os.cpu_count()

In [ ]:
import threading

class Master:
    def __init__(self, condition, event):
        self.condition = condition
        self.event = event

    def run(self):
        for _ in range(10):
            with self.condition:
                print("Maître : J'attends que les agents soient prêts.")
                self.condition.wait()
                print("Maître : Autorisation d'exécuter le travail.")
                self.event.set()
                self.condition.notify_all()

class Agent:
    def __init__(self, agent_id, condition, event):
        self.agent_id = agent_id
        self.condition = condition
        self.event = event

    def run(self):
        for _ in range(10):
            with self.condition:
                print(f"Agent {self.agent_id} : Prêt à travailler.")
                self.condition.notify()
                self.condition.wait()
                self.event.wait()
                self.event.clear()
                print(f"Agent {self.agent_id} : Travail en cours.")
                self.condition.notify()
                self.condition.wait()
                print(f"Agent {self.agent_id} : Travail terminé.")

# Créez une Condition pour coordonner les agents et le maître
condition = threading.Condition()

# Créez un Event pour autoriser les agents à travailler
event = threading.Event()

# Créez le maître et les agents
master = Master(condition, event)
agents = [Agent(i, condition, event) for i in range(1, 4)]

# Créez des threads pour le maître et les agents
master_thread = threading.Thread(target=master.run)
agent_threads = [threading.Thread(target=agent.run) for agent in agents]

# Démarrez les threads
master_thread.start()
for thread in agent_threads:
    thread.start()

# Attendez que tous les threads se terminent
master_thread.join()
for thread in agent_threads:
    thread.join()

print("Tous les agents et le maître ont terminé.")


In [ ]:
import time
from threading import Thread
from queue import Queue

class Agent:
    def __init__(self, master_queue, name):
        self.master_queue = master_queue
        self.name = name

    def do_task(self, task):
        print(f"Agent {self.name} received task: {task}")
        # Simulating task execution
        time.sleep(2)
        print(f"Agent {self.name} completed task: {task}")
        # Inform the master that the task is completed
        self.master_queue.put((self.name, task))


class Master:
    def __init__(self):
        self.agent_queues = {}

    def add_agent(self, agent):
        agent_queue = Queue()
        self.agent_queues[agent.name] = agent_queue
        Thread(target=self.listen_to_agent, args=(agent, agent_queue)).start()

    def assign_task(self, agent_name, task):
        agent_queue = self.agent_queues.get(agent_name)
        if agent_queue:
            print(f"Master assigned task: {task} to Agent {agent_name}")
            agent_queue.put(task)
        else:
            print(f"Agent {agent_name} not found.")

    def listen_to_agent(self, agent, agent_queue):
        while True:
            task = agent_queue.get()
            if task is None:
                break
            print(f"Master received task completion report from Agent {agent.name} for task: {task}")


# Exemple d'utilisation
if __name__ == "__main__":
    master = Master()

    agents = [Agent(master_queue=master, name=f"Agent{i+1}") for i in range(4)]

    for agent in agents:
        master.add_agent(agent)

    master.assign_task("Agent1", "Task1")
    master.assign_task("Agent2", "Task2")
    master.assign_task("Agent3", "Task3")
    master.assign_task("Agent4", "Task4")

    # Attendez la fin des tâches avant de quitter
    time.sleep(3)  # Simuler le temps nécessaire pour terminer les tâches
    for agent in agents:
        master.add_agent(agent)  # Envoyer un signal de fin à tous les agents


In [ ]:
import multiprocessing
import time
import random

class Agent(multiprocessing.Process):
    def __init__(self, agent_id, master_queue, agent_queue):
        super().__init__()
        self.agent_id = agent_id
        self.master_queue = master_queue
        self.agent_queue = agent_queue

    def run(self):
        while True:
            instruction = self.master_queue.get()

            if instruction == "QUIT":
                break

            # Ajoutez des conditions en fonction de l'instruction
            if instruction == "Commande 1":
                result = f"Agent {self.agent_id} exécute la Commande 1 spécifique."
            elif instruction == "Commande 2":
                result = f"Agent {self.agent_id} exécute la Commande 2 spécifique."
            else:
                result = f"Agent {self.agent_id} exécute une instruction non gérée."

            self.agent_queue.put(result)

class Master:
    def __init__(self):
        self.master_queue = multiprocessing.Queue()
        self.agent_queues = [multiprocessing.Queue() for _ in range(4)]
        self.agents = [Agent(i, self.master_queue, self.agent_queues[i]) for i in range(4)]

    def start_agents(self):
        for agent in self.agents:
            agent.start()

    def send_instructions(self, instructions):
        for instruction in instructions:
            for agent_queue in self.agent_queues:
                self.master_queue.put(instruction)

            time.sleep(1)  # Attendez un certain temps pour simuler le traitement parallèle

            for _ in range(4):
                report = self.master_queue.get()
                print(report)

    def stop_agents(self):
        for _ in range(4):
            self.master_queue.put("QUIT")

        for agent in self.agents:
            agent.join()

if __name__ == "__main__":
    master_instance = Master()
    master_instance.start_agents()

    instructions = ["Commande 1", "Commande 2", "Commande 3", "Commande 4"]
    master_instance.send_instructions(instructions)

    master_instance.stop_agents()


In [ ]:
import multiprocessing
import time

class Agent(multiprocessing.Process):
    def __init__(self, agent_id, master_event, agent_queue):
        super().__init__()
        self.agent_id = agent_id
        self.master_event = master_event
        self.agent_queue = agent_queue

    def run(self):
        while True:
            # Wait for the event to be set by the master
            self.master_event.wait()

            instruction = self.agent_queue.get()

            if instruction == "QUIT":
                break

            # Your conditions or logic here
            result = f"Agent {self.agent_id} a exécuté l'instruction: {instruction}"

            # Report the result to the master
            self.agent_queue.put(result)

class Master:
    def __init__(self):
        self.master_event = multiprocessing.Event()
        self.agent_queues = [multiprocessing.Queue() for _ in range(4)]
        self.agents = [Agent(i, self.master_event, self.agent_queues[i]) for i in range(4)]

    def start_agents(self):
        for agent in self.agents:
            agent.start()

    def send_instructions(self, instructions):
        for instruction in instructions:
            # Send the instruction to all agents
            for agent_queue in self.agent_queues:
                agent_queue.put(instruction)

            # Set the event to signal agents to start processing
            self.master_event.set()

            time.sleep(1)  # Simulate parallel processing time

            # Clear the event for the next round
            self.master_event.clear()

            # Collect reports from agents
            for _ in range(4):
                report = self.agent_queues[0].get()  # Assuming all agents provide the same result
                print(report)

    def stop_agents(self):
        # Send the QUIT instruction to all agents
        for agent_queue in self.agent_queues:
            agent_queue.put("QUIT")

        # Wait for agents to finish
        for agent in self.agents:
            agent.join()

if __name__ == "__main__":
    master_instance = Master()
    master_instance.start_agents()

    instructions = ["Commande 1", "Commande 2", "Commande 3", "Commande 4"]
    master_instance.send_instructions(instructions)

    master_instance.stop_agents()
